# Naive Bayes - Trabalho

## Questão 1

Implemente um classifacor Naive Bayes para o problema de predizer a qualidade de um carro. Para este fim, utilizaremos um conjunto de dados referente a qualidade de carros, disponível no [UCI](https://archive.ics.uci.edu/ml/datasets/car+evaluation). Este dataset de carros possui as seguintes features e classe:

** Attributos **
1. buying: vhigh, high, med, low
2. maint: vhigh, high, med, low
3. doors: 2, 3, 4, 5, more
4. persons: 2, 4, more
5. lug_boot: small, med, big
6. safety: low, med, high

** Classes **
1. unacc, acc, good, vgood

## Questão 2
Crie uma versão de sua implementação usando as funções disponíveis na biblioteca SciKitLearn para o Naive Bayes ([veja aqui](http://scikit-learn.org/stable/modules/naive_bayes.html)) 

## Questão 3

Analise a acurácia dos dois algoritmos e discuta a sua solução.

# Resolução

## Questão 1


In [1]:
# X = [0.5, 0.6, 0.7, 1, 2, 3] => predizer a classe
# 
# p(c|x) = P(X|C)*P(C) / P(X) => P(X|C)*P(C)
# MAIOR P(C|X) ai a gente retorna que é a c

# probabilidade da classe = p(c) => len(classe)/numero de linhas do dataset
# probabilidade do elemento x dado a classe = p(x|c) = p(x[0]|c)*p(x[1]|c)*p(x[2]|c)*...*p(x[5]|c)  
# X = [0.5, 0.6, 0.7, 1, 2, 3]
# probabilidade do elemento x = p(x)

# probabilidade de cada elemento para cada feature para cada classe => elemento/len(classe)
# os valores da classe estão ordenados de 0 a 3

In [7]:
import numpy as np
import pandas as pd
import random
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.naive_bayes import GaussianNB
from statistics import mean
    
def categorize_data(dataset, columns):
    # Categorizando os dados
    for feature in columns:
        dataset[feature] = dataset[feature].astype('category')
        dataset[feature] = dataset[feature].cat.codes
    return dataset

def summarize_by_class(dataset):
    classes = [[], [], [], []]
    # separando por classe
    for row in dataset['class'].unique():
        classes[row].append(dataset[dataset['class'] == row])
    return classes

def calculate_probabilities(dataset, features, classes): 
    #separando cada valor de cada feature pra cada classe
    classes_unique = [[], [], [], []]
    classes_feature_value = [[], [], [], []]
    probs = [[], [], [], []]
    for i in range(0, len(classes)):  
        df = classes[i][0]
        for ft in features:   
            # Classes_unique vai ser usado para mapear os valores direito, já que não estão ordenados
            classes_unique[i].append(df[ft].unique())
            classes_feature_value[i].append(df[ft].unique())
            probs[i].append(df[ft].unique())

    # Gerando a contagem de cada elemento de cada feature para cada classe 
    # Percorre as classes
    for i in range(0, len(classes)):  
        df = classes[i][0]
        # Percorre as features dentro da classe
        for ft in range(0, len(features)):
            # Percorrer os valores de dentro das features
            for col in range(0, len(classes_unique[i][ft])):
                # isso aqui eu pego o valor do elemento que a feture ft pode ter
                elemento = classes_unique[i][ft][col]
                # Pego o tamanho de linhas do dataset retornado por essa consulta em que o valor da linha para o atributo = elemento
                classes_feature_value[i][ft][col] = len(df[df[features[ft]] == elemento])
                probs[i][ft][col] = len(df[df[features[ft]] == elemento])

    # Geração dos valores de probabilidades de cada valor de cala feature
    for i in range(0, len(classes)):  
        df = classes[i][0]
        for ft in range(0, len(features)):
            probs[i][ft] = probs[i][ft].astype(float)
            for col in range(0, len(classes_unique[i][ft])):
                # isso aqui eu pego o valor do elemento que a feture ft pode ter
                # Pego o tamanho de linhas do dataset retornado por essa consulta em que o valor da linha para o atributo = elemento
                probs[i][ft][col] = probs[i][ft][col].astype(float)/sum(classes_feature_value[i][ft])

    probabilidade_classes = [len(classe[0])/len(dataset) for classe in classes]
    
    return probs, probabilidade_classes
##################################################### 
def predict(X, features, probs, probabilidade_classes, classes_unique):
    results = []
    # Ajeitar essa última parte que tá dando erro, mas a lógica tá correta
    # Documentar todo esse código
    for classe in range(0, len(probs)):
        produtorio = 1
        for feature in range(0, len(features)):
            posix = 0
            boolean = False
            print(feature)
            print(classes_unique[classe])
            break
#             for i in range(0, len(classes_unique[classe][feature])):
#                 if(X[feature] == classes_unique[classe][feature][i]):
#                     posix = i
#                     boolean = True
#                     break
            if(boolean):
                produtorio *= probs[classe][feature][posix]
        results.append(produtorio)

    for i in range(0, len(results)):
        results[i] = results[i] * probabilidade_classes[i]
    
    classes_real_oficial = 0
    for i in range(0, len(results)):
        if results[i] == max(results):
            classes_real_oficial = i
    return classes_real_oficial

def get_predict(dados, features, probs, probabilidade_classes, classes_unique):
    predicts = []
    for i in range(0, len(dados)):
        data = np.array(dados.loc[dados.index[[i]]])[0]
        predicts.append(predict(data, features, probs, probabilidade_classes, classes_unique))
    return predicts

def accuracy(predicts, teste_y):
    cont = 0
    for i in range(0, len(predicts)):
        teste = np.array(teste_y.loc[teste_y.index[[i]]])[0]
        if(predicts[i] == teste):
            cont += 1
    return 100*(cont/len(predicts))
        

In [8]:
columns = ["buying", "maint", "doors", "persons", "lug_boot", "safety", "class"]
features = ["buying", "maint", "doors", "persons", "lug_boot", "safety"]
map_class = {0:'acc', 1:'good', 2:'unacc', 3:'vgood'}

dataset = pd.read_csv("carData.csv", names=["buying", "maint", "doors", "persons", "lug_boot", "safety", "class"])
dataset = categorize_data(dataset, columns)

treino, teste = train_test_split(dataset,test_size=0.3)
treino_x = treino[features]
treino_y = treino['class']
teste_x = teste[features]
teste_y = teste['class']

classes = summarize_by_class(treino)
tabela_probabilidades, probabilidade_classes = calculate_probabilities(treino, features, classes)
# predicts = get_predict(teste_x, features, tabela_probabilidades, probabilidade_classes, treino_y.unique())
# print(accuracy(predicts, teste_y))

In [9]:
def createNB(x, y):
    classes = np.unique(y)
    K = classes.shape[0]
    D = x.shape[1]
    
    prioridadeClasses = np.array([np.sum(y == k) for k in classes])/y.shape[0]
    
    mean = np.zeros((D, K))
    var = np.zeros((D, K))
    for k in classes:
        xk = x[y == k]
        mean[:,k] = np.mean(xk, axis = 0)
        var[:, k] = np.sum((xk - mean[:, k])**2, axis = 0) / xk.shape[0]
    
    return {'classes': classes, 'prioridade': prioridadeClasses, 'K': K, 'D': D, 'mean': mean, 'var':var}

def predic(model, x):
    pred = np.zeros((x.shape[0], model['K']))
    
    for k in range(model['K']):
        pred[:,k] = - 0.5 * np.sum(np.log(2*np.pi*model['var'][:,k])) \
                    - 0.5 * np.sum((x - model['mean'][:,k])**2 / model['var'][:,k], axis=1) \
                    + model['prioridade'][k]
    return model['classes'][np.argmax(pred, axis=1)]

In [10]:
model = createNB(np.array(treino_x), np.array(treino_y))
model
predict = predic(model, teste_x)
# classe
accuracy_score(teste_y, predict)


C:\Users\andre\Miniconda3\lib\site-packages\ipykernel_launcher.py:23: RuntimeWarning: divide by zero encountered in log


0.03468208092485549

# Questão 2

In [11]:
for feature in ["buying", "maint", "doors", "persons", "lug_boot", "safety", "class"]:
    dataset[feature] = dataset[feature].astype('category')
    dataset[feature] = dataset[feature].cat.codes

kf = KFold(n_splits = 5, shuffle = True)
accuracy1 = [];
accuracy2 = [];

for _ in range(0,5):
    result = next(kf.split(dataset), None)
    train = dataset.iloc[result[0]]
    test =  dataset.iloc[result[1]]
    
    target_train = train['class']
    target_test = test['class']
    
    train = train[["buying", "maint", "doors", "persons", "lug_boot", "safety"]]
    test = test[["buying", "maint", "doors", "persons", "lug_boot", "safety"]]
    
    naive_bayes = GaussianNB()
    naive_bayes.fit(train, target_train)
    target_pred = naive_bayes.predict(test)
    
    model = createNB(np.array(train), np.array(target_train))
    predict = predic(model, test)
    
    accuracy1.append(accuracy_score(target_test, target_pred))
    accuracy2.append(accuracy_score(target_test, predict))

print(accuracy1)
print(accuracy2)
# Fazer média da acuracia
print("\nClassification Report:")
print(classification_report(target_pred, target_test))
print("--------------------------------------------------")
print(classification_report(predict, target_test))

C:\Users\andre\Miniconda3\lib\site-packages\ipykernel_launcher.py:23: RuntimeWarning: divide by zero encountered in log
C:\Users\andre\Miniconda3\lib\site-packages\ipykernel_launcher.py:23: RuntimeWarning: divide by zero encountered in log
C:\Users\andre\Miniconda3\lib\site-packages\ipykernel_launcher.py:23: RuntimeWarning: divide by zero encountered in log
C:\Users\andre\Miniconda3\lib\site-packages\ipykernel_launcher.py:23: RuntimeWarning: divide by zero encountered in log


[0.6358381502890174, 0.6040462427745664, 0.6358381502890174, 0.6040462427745664, 0.661849710982659]
[0.02601156069364162, 0.04046242774566474, 0.04335260115606936, 0.046242774566473986, 0.028901734104046242]

Classification Report:
              precision    recall  f1-score   support

           0       0.08      0.38      0.14        16
           1       0.00      0.00      0.00         0
           2       0.85      0.86      0.85       248
           3       1.00      0.12      0.22        82

    accuracy                           0.66       346
   macro avg       0.48      0.34      0.30       346
weighted avg       0.85      0.66      0.67       346

--------------------------------------------------
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.00      0.00      0.00         0
           2       0.00      0.00      0.00         0
           3       1.00      0.03      0.06       346

    accura

C:\Users\andre\Miniconda3\lib\site-packages\ipykernel_launcher.py:23: RuntimeWarning: divide by zero encountered in log
C:\Users\andre\Miniconda3\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


# Questão 3

